In [1]:
import numpy as np
import pandas as pd

### dataset, cur_fold, r, lr, num_iter= sys.argv[1:]


In [2]:
err = {}
for dataset in [1, 2]:
    err[dataset] = {}
    for cur_fold in range(5):
        err[dataset][cur_fold] = {}
        for r in range(1 ,21):
            err[dataset][cur_fold][r] = {}
            for lr in [0.01, 0.1, 1.0, 2.0]:
                err[dataset][cur_fold][r][lr] = {}
                for num_iter in range(100, 2600, 400):
                    err[dataset][cur_fold][r][lr][num_iter] = \
                    np.load("../code/baseline/stf-nested/stf-error-{}-{}-{}-{}-{}.npy".format(dataset, cur_fold, r, lr, num_iter)).item()

In [5]:
iter = pd.DataFrame(err[1][0][3][0.01]).mean().idxmin()

In [6]:
pd.DataFrame(err[1][0][3][0.01]).mean()[2500]

177.28915755534743

In [3]:
best_r = {}
best_lr = {}
best_iter = {}
for dataset in [1, 2]:
    best_r[dataset] = {}
    best_lr[dataset] = {}
    best_iter[dataset] = {}
    for cur_fold in range(5):
        min_error = np.inf
        for r in range(1, 21):
            for lr in [0.01, 0.1, 1.0, 2.0]:
                iter = pd.DataFrame(err[dataset][cur_fold][r][lr]).mean().idxmin()
                error = pd.DataFrame(err[dataset][cur_fold][r][lr]).mean()[iter]
                if error < min_error:
                    min_error = error
                    best_r[dataset][cur_fold] = r
                    best_lr[dataset][cur_fold] = lr
                    best_iter[dataset][cur_fold] = iter

In [4]:
pd.DataFrame(best_r)

,1,2
0,20,16
1,18,13
2,17,13
3,10,3
4,20,15


In [5]:
pd.DataFrame(best_lr)

,1,2
0,2.0,1.0
1,1.0,2.0
2,2.0,0.1
3,2.0,2.0
4,1.0,1.0


In [6]:
pd.DataFrame(best_iter)

,1,2
0,100,1700
1,1700,500
2,100,2500
3,2100,100
4,2500,2100


In [7]:
import sys
sys.path.append("../code/")
from dataloader import APPLIANCE_ORDER, get_train_test
from tensor_custom_core import stf_4dim, stf_4dim_time


In [8]:
def stf(dataset, cur_fold, r=2, lr=1, num_iter=100):
    num_folds=5
    train, test = get_train_test(dataset, num_folds=num_folds, fold_num=cur_fold)
    test_copy = test.copy()
    test_copy[:, 1:, :, :] = np.NaN
    train_test = np.concatenate([train, test_copy])
    H, A, D, T = stf_4dim(tensor=train_test, r=r, lr=lr, num_iter=num_iter)
    pred = np.einsum("Hr, Ar, Dr, Tr ->HADT", H, A, D, T)[len(train):, 1:, :, :]
    
    return pred

### dataset1

In [ ]:
dataset = 1
out = []
num_fold=5
for cur_fold in range(5):
    b_r= best_r[dataset][cur_fold]
    b_lr = best_lr[dataset][cur_fold]
    b_iter = best_iter[dataset][cur_fold]
    
    pred = stf(dataset, cur_fold, b_r, b_lr, b_iter)
    
    out.append(pred)

798.933843056 0
378.803348099 50
774.407629853 0
365.00344319 50
348.621090837 100


In [24]:
cont_out = np.concatenate(out)
tensor = np.load('../2015-5appliances.numpy.npy')
gt = tensor[:, 1:, :, :]

In [26]:
from sklearn.metrics import mean_absolute_error
err_stf = {APPLIANCE_ORDER[i+1]:mean_absolute_error(cont_out[:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(pred.shape[1])}

In [27]:
err_stf

{'dr': 117.55267578113788,
 'dw': 27.461522878707523,
 'fridge': 43.819964033418202,
 'hvac': 331.63421940163073,
 'mw': 10.488311233484415}

### dataset 2

In [ ]:
dataset = 2
out = []
num_fold=5
for cur_fold in range(5):
    b_r= best_r[dataset][cur_fold]
    b_lr = best_lr[dataset][cur_fold]
    b_iter = best_iter[dataset][cur_fold]
    
    pred = stf(dataset, cur_fold, b_r, b_lr, b_iter)
    
    out.append(pred)

1252.33308992 0
527.959217355 50
497.913017577 100
485.598775016 150
480.795097588 200
478.138420724 250
476.488134263 300
475.32242377 350
474.349240402 400
473.447005509 450
472.608111016 500
471.894729038 550
471.32460406 600
470.90972481 650
470.597483214 700
470.359519399 750
470.173643753 800
470.025461654 850
469.903689551 900
470.2406338 950


In [19]:
cont_out_2 = np.concatenate(out)
tensor = np.load('../2015-5appliances-true-agg.npy')
gt = tensor[:, 1:, :, :]

In [20]:
from sklearn.metrics import mean_absolute_error
err_stf = {APPLIANCE_ORDER[i+1]:mean_absolute_error(cont_out_2[:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(cont_out_2.shape[1])}

In [21]:
err_stf

{'dr': 116.04920935644708,
 'dw': 25.042311604862935,
 'fridge': 42.772340749615211,
 'hvac': 438.29128414467749,
 'mw': 16.192845365806502}